In [1]:
from gensim import corpora, models
import pandas as pd
import numpy as np
import sys
from collections import Counter
import itertools
from lsi_tagger.model import TagExtractor

%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\David\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
df = pd.read_csv('data/h&m_kaggle_products.csv')
df = df[~df.isnull().any(axis=1)]
print(df.shape)
df.head(2)

(105126, 25)


,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.


In [3]:
documents = df['detail_desc'].drop_duplicates().values

te = TagExtractor(word_count_min=5, word_length_min=2, num_lsi_topics=200, 
                  bigram_kwargs={'bigrams_pmi_min_value':1, 'bigrams_min_freq':200})
te.fit(documents)

Filtering tokens: 100%|████████████████████████████████████████████████████████| 43404/43404 [00:11<00:00, 3653.18it/s]
Making bigrams: 43404it [00:00, 59723.28it/s]
Filtering by word count: 100%|████████████████████████████████████████████████| 43404/43404 [00:00<00:00, 75380.44it/s]
2022-04-18 21:55:25 INFO     Training TF-IDF...
2022-04-18 21:55:25 INFO     adding document #0 to Dictionary(0 unique tokens: [])
2022-04-18 21:55:25 INFO     adding document #10000 to Dictionary(1700 unique tokens: ['jersey', 'jersey top', 'narrow', 'narrow shoulder', 'shoulder']...)
2022-04-18 21:55:26 INFO     adding document #20000 to Dictionary(1869 unique tokens: ['jersey', 'jersey top', 'narrow', 'narrow shoulder', 'shoulder']...)
2022-04-18 21:55:27 INFO     adding document #30000 to Dictionary(1979 unique tokens: ['jersey', 'jersey top', 'narrow', 'narrow shoulder', 'shoulder']...)
2022-04-18 21:55:27 INFO     adding document #40000 to Dictionary(2035 unique tokens: ['jersey', 'jersey top', 'nar

In [4]:
len(te.tc.bigrams_dict)

200

In [5]:
tags = te.transform(input_document='Jersey top with narrow shoulder straps.',
                    candidate_documents=[
                        'Tights with built-in support to lift the bottom. Black in 30 denier and light amber in 15 denier.',
                        'Semi shiny tights that shape the tummy, thighs and calves while also encouraging blood circulation in the legs. Elasticated waist.',
                        'Jersey top with narrow shoulder straps.',
                        'uygnhjkl'
                    ], 
                    n_input_tags=10, n_candidate_tags=10, n_adjacent_tags=10)

tags

([('jersey top', 1),
  ('narrow shoulder', 1),
  ('shoulder straps', 1),
  ('shoulder', 1),
  ('narrow', 1),
  ('straps', 1),
  ('top', 1),
  ('jersey', 1)],
 [{},
  {},
  {'jersey top': 0.8110500443667185,
   'narrow shoulder': 0.787473373800416,
   'shoulder straps': 0.6065702133242676,
   'shoulder': 0.6043741598461158,
   'narrow': 0.5937411700150707,
   'straps': 0.5781561715575201,
   'top': 0.39426017782089023,
   'jersey': 0.3605001445282605},
  {}],
 ['join',
  'stitched',
  'swimsuits',
  'knots',
  'tankini',
  'stripe-print',
  'cross',
  'fully-lined',
  'cold',
  'backless'])

In [6]:
new_candidate_ranking = te.rank(candidate_tags=tags[1], selected_tags=['jersey'])
new_candidate_ranking

[2, 0, 1, 3]

In [7]:
te.save()
te = TagExtractor()
te.load()

In [8]:
tags = te.transform(input_document='Jersey top with narrow shoulder straps.',
                    candidate_documents=[
                        'Tights with built-in support to lift the bottom. Black in 30 denier and light amber in 15 denier.',
                        'Semi shiny tights that shape the tummy, thighs and calves while also encouraging blood circulation in the legs. Elasticated waist.',
                        'Jersey top with narrow shoulder straps.',
                        'uygnhjkl'
                    ], 
                    n_input_tags=10, n_candidate_tags=10, n_adjacent_tags=10)

tags

([('jersey top', 1),
  ('narrow shoulder', 1),
  ('shoulder straps', 1),
  ('shoulder', 1),
  ('narrow', 1),
  ('straps', 1),
  ('top', 1),
  ('jersey', 1)],
 [{},
  {},
  {'jersey top': 0.8110500443667185,
   'narrow shoulder': 0.787473373800416,
   'shoulder straps': 0.6065702133242676,
   'shoulder': 0.6043741598461158,
   'narrow': 0.5937411700150707,
   'straps': 0.5781561715575201,
   'top': 0.39426017782089023,
   'jersey': 0.3605001445282605},
  {}],
 ['join',
  'stitched',
  'swimsuits',
  'knots',
  'tankini',
  'stripe-print',
  'cross',
  'fully-lined',
  'cold',
  'backless'])

In [9]:
new_candidate_ranking = te.rank(candidate_tags=tags[1], selected_tags=['jersey'])
new_candidate_ranking

[2, 0, 1, 3]

In [14]:
from tqdm import tqdm

class ContentRecommender:
    def __init__(self):
        lsi_vecs = np.array([te._lsi_corpus2vec(cl) for cl in tqdm(te.corpus_lsi, desc='Creating LSI Vectors', position=0)])
        self.lsi_vecs_normed = lsi_vecs/np.linalg.norm(lsi_vecs, axis=1)[:,None]
    
    def _show(self, candidate_documents, candidate_tags):
        print("#########################")
        print("######### INPUTS ########")
        print("#########################\n")
        print(f"Input Document: {self.input_document}")
        print(f"\nAdjacent Tags: {self.adjacent_tags}")
        print(f"\nInput Tags: {self.input_tags}")
        
        print("\n#########################")
        print("#### RECOMMENDATIONS ####")
        print("#########################\n")
        for n, candidate_doc in enumerate(candidate_documents):
            print(f"{n+1}. {candidate_doc}")
            display(candidate_tags[n])
            print('-'*150)
            if (n+1) == self.show_n_recs:
                break
                
    def _get_content_recommendations(self, input_document):
        _, input_corpus_lsi = te._get_vector_representations(input_document)
        input_lsi_vec = te._lsi_corpus2vec(input_corpus_lsi)
        input_lsi_vec_normed = input_lsi_vec/np.linalg.norm(input_lsi_vec)
        sims = input_lsi_vec_normed.dot(self.lsi_vecs_normed.T)
        top_inds = np.argsort(sims)[::-1][:(self.internal_n_recs+1)]
        self.candidate_documents = [documents[ti] for ti in top_inds if documents[ti]!=input_document]
        tags = te.transform(input_document=input_document,
                            candidate_documents=self.candidate_documents, 
                            n_input_tags=self.n_input_tags, 
                            n_candidate_tags=self.n_candidate_tags,
                            n_adjacent_tags=self.n_adjacent_tags)
        self.input_tags, self.candidate_tags, self.adjacent_tags = tags
    
    def initial_recommendations(self, input_document, 
                                internal_n_recs=50, show_n_recs=5,
                                n_input_tags=10, n_candidate_tags=5,
                                n_adjacent_tags=10):
        self.__dict__.update(locals())
        self._get_content_recommendations(self.input_document)
        self._show(self.candidate_documents, self.candidate_tags)
        
    def rerank(self, selected_tags):
        ranking = te.rank(candidate_tags=self.candidate_tags, 
                          selected_tags=selected_tags)
        candidate_documents = np.array(self.candidate_documents)[ranking]
        candidate_tags = np.array(self.candidate_tags)[ranking]
        self._show(candidate_documents, candidate_tags)

In [15]:
recommender = ContentRecommender()

Creating LSI Vectors: 100%|████████████████████████████████████████████████████| 43404/43404 [00:16<00:00, 2569.83it/s]


In [16]:
# Use an existing product description as the input, to find similar products
recommender.initial_recommendations(input_document=documents[10000], 
                                    internal_n_recs=50, show_n_recs=5,
                                    n_input_tags=10, n_candidate_tags=5, 
                                    n_adjacent_tags=10)

#########################
######### INPUTS ########
#########################

Input Document: Straight-cut, linen-blend jacket in a patterned weave with a shawl collar, padded shoulders and long sleeves with decorative buttons at the cuffs. Buttons at the front and front pockets. Lined.

Adjacent Tags: ['single-breasted', 'two-button', 'lapel', 'stitching', 'taper', 'buttonhole', 'narrower', 'peak', 'creates', 'combined']

Input Tags: [('buttons', 49), ('decorative', 45), ('jacket', 44), ('decorative buttons', 42), ('straight-cut', 26), ('front', 19), ('front pockets', 8), ('long sleeves', 7), ('long', 4), ('patterned', 3)]

#########################
#### RECOMMENDATIONS ####
#########################

1. Straight-cut, double-breasted jacket in a patterned weave with notch lapels and covered buttons at the front. Long sleeves with decorative buttons at the cuffs and welt front pockets with a flap. Lined.


{'buttons': 0.6219453826622374,
 'straight-cut': 0.5223011030807179,
 'decorative': 0.43807280817399746,
 'jacket': 0.3927226941621633,
 'decorative buttons': 0.3726572577164249}

------------------------------------------------------------------------------------------------------------------------------------------------------
2. Single-breasted jacket in a viscose weave with peak lapels, jetted front pockets and decorative buttons at the cuffs. Lined.


{'buttons': 0.5265288364249219,
 'decorative': 0.4767221001553202,
 'jacket': 0.4029517441561594,
 'decorative buttons': 0.3755788549005063,
 'front': 0.3125079199674479}

------------------------------------------------------------------------------------------------------------------------------------------------------
3. Straight-cut jacket in a patterned linen and viscose weave with notch lapels, buttons at the front, welt front pockets and decorative buttons at the cuffs. Partly lined.


...

------------------------------------------------------------------------------------------------------------------------------------------------------
4. Double-breasted jacket in a striped weave with peak lapels, jetted front pockets and decorative buttons at the cuffs. Lined.


{'buttons': 0.5104773418875925,
 'decorative': 0.459140563092844,
 'jacket': 0.38794845692571633,
 'decorative buttons': 0.35220942844455766,
 'front': 0.31192712336377276}

------------------------------------------------------------------------------------------------------------------------------------------------------
5. Straight-cut, double-breasted jacket in woven fabric with peak lapels, welt front pockets and decorative buttons at the cuffs. Lined.


{'straight-cut': 0.542402080298982,
 'buttons': 0.5162499523607565,
 'decorative': 0.4594577045477072,
 'jacket': 0.39636234014421357,
 'decorative buttons': 0.3609180794865859}

------------------------------------------------------------------------------------------------------------------------------------------------------


In [17]:
# Re-rank product recommendations with emphasis on a few input tags
recommender.rerank(selected_tags=['patterned'])

#########################
######### INPUTS ########
#########################

Input Document: Straight-cut, linen-blend jacket in a patterned weave with a shawl collar, padded shoulders and long sleeves with decorative buttons at the cuffs. Buttons at the front and front pockets. Lined.

Adjacent Tags: ['single-breasted', 'two-button', 'lapel', 'stitching', 'taper', 'buttonhole', 'narrower', 'peak', 'creates', 'combined']

Input Tags: [('buttons', 49), ('decorative', 45), ('jacket', 44), ('decorative buttons', 42), ('straight-cut', 26), ('front', 19), ('front pockets', 8), ('long sleeves', 7), ('long', 4), ('patterned', 3)]

#########################
#### RECOMMENDATIONS ####
#########################

1. Straight-cut shirt in a soft, patterned viscose weave with a collar and buttons down the front. Long sleeves with narrow, buttoned cuffs, and a rounded hem.


{'straight-cut': 0.6326522963543335,
 'buttons': 0.5350280617476866,
 'patterned': 0.4667432102831318,
 'long sleeves': 0.37022598064262846,
 'long': 0.3368321783200726}

------------------------------------------------------------------------------------------------------------------------------------------------------
2. Short, straight-cut coat in a patterned weave with a zip down the front, long sleeves and zipped front pockets with a fake flap. Lined.


{'straight-cut': 0.6214405012240144,
 'patterned': 0.430498938127809,
 'long sleeves': 0.3242311857307232,
 'front': 0.32028712033860496,
 'pockets': 0.3142584746365671}

------------------------------------------------------------------------------------------------------------------------------------------------------
3. Jacket in a linen and cotton weave with notch lapels, buttons at the front, a chest pocket and flap front pockets. Decorative buttons at the cuffs and a single back vent. Patterned lining.


...

------------------------------------------------------------------------------------------------------------------------------------------------------
4. Straight-cut, double-breasted jacket in a patterned weave with notch lapels and covered buttons at the front. Long sleeves with decorative buttons at the cuffs and welt front pockets with a flap. Lined.


{'buttons': 0.6219453826622374,
 'straight-cut': 0.5223011030807179,
 'decorative': 0.43807280817399746,
 'jacket': 0.3927226941621633,
 'decorative buttons': 0.3726572577164249}

------------------------------------------------------------------------------------------------------------------------------------------------------
5. Single-breasted jacket in a viscose weave with peak lapels, jetted front pockets and decorative buttons at the cuffs. Lined.


{'buttons': 0.5265288364249219,
 'decorative': 0.4767221001553202,
 'jacket': 0.4029517441561594,
 'decorative buttons': 0.3755788549005063,
 'front': 0.3125079199674479}

------------------------------------------------------------------------------------------------------------------------------------------------------


In [18]:
# Use free-text to find products that match the description best
recommender.initial_recommendations(input_document='leather gloves', 
                                    internal_n_recs=50, show_n_recs=5,
                                    n_input_tags=10, n_candidate_tags=5, 
                                    n_adjacent_tags=10)

#########################
######### INPUTS ########
#########################

Input Document: leather gloves

Adjacent Tags: ['fingerless', 'uppers', 'index', 'touchscreen-compatible', 'thumb', 'finger', 'separately', 'reveal', 'palms', 'wool-blend']

Input Tags: [('leather', 38), ('gloves', 32)]

#########################
#### RECOMMENDATIONS ####
#########################

1. Gloves in soft, supple leather. Lined.


{'leather': 0.2740454135597106, 'gloves': 0.17103218138615325}

------------------------------------------------------------------------------------------------------------------------------------------------------
2. Gloves in soft, supple imitation leather. Lined.


{'leather': 0.4110272996398084, 'gloves': 0.1639349647009004}

------------------------------------------------------------------------------------------------------------------------------------------------------
3. Knitted gloves with imitation leather uppers and palms, and foldover cuffs. Fleece lining.


...

------------------------------------------------------------------------------------------------------------------------------------------------------
4. Gloves in soft leather with a wool-blend lining.


{'leather': 0.2607548376302656, 'gloves': 0.17635336392652126}

------------------------------------------------------------------------------------------------------------------------------------------------------
5. Gloves in soft, supple leather with a zip at the top. Lined.


{'leather': 0.2736426138830019, 'gloves': 0.17029411595515925}

------------------------------------------------------------------------------------------------------------------------------------------------------


In [19]:
# Use adjacent tags to make a different query
recommender.initial_recommendations(input_document='touchscreen-compatible leather gloves', 
                                    internal_n_recs=50, show_n_recs=5,
                                    n_input_tags=10, n_candidate_tags=5, 
                                    n_adjacent_tags=10)

#########################
######### INPUTS ########
#########################

Input Document: touchscreen-compatible leather gloves

Adjacent Tags: ['fingerless', 'uppers', 'index', 'thumb', 'finger', 'separately', 'reveal', 'palms', 'wool-blend', 'wrists']

Input Tags: [('gloves', 37), ('leather', 34), ('touchscreen-compatible', 4)]

#########################
#### RECOMMENDATIONS ####
#########################

1. Gloves in soft, supple leather. Lined.


{'leather': 0.21454706074265623, 'gloves': 0.13979451376818156}

------------------------------------------------------------------------------------------------------------------------------------------------------
2. Knitted gloves with imitation leather uppers and palms, and foldover cuffs. Fleece lining.


{'leather': 0.2732140068110509, 'gloves': 0.165792919760358}

------------------------------------------------------------------------------------------------------------------------------------------------------
3. Gloves in soft, supple imitation leather. Lined.


...

------------------------------------------------------------------------------------------------------------------------------------------------------
4. Gloves in soft leather with a wool-blend lining.


{'leather': 0.20125648481321118, 'gloves': 0.14511569630854956}

------------------------------------------------------------------------------------------------------------------------------------------------------
5. Gloves in soft, supple leather with a zip at the top. Lined.


{'leather': 0.21414426106594753, 'gloves': 0.13905644833718755}

------------------------------------------------------------------------------------------------------------------------------------------------------


In [20]:
# Re-rank product recommendations based on an input tag
recommender.rerank(selected_tags=['touchscreen-compatible'])

#########################
######### INPUTS ########
#########################

Input Document: touchscreen-compatible leather gloves

Adjacent Tags: ['fingerless', 'uppers', 'index', 'thumb', 'finger', 'separately', 'reveal', 'palms', 'wool-blend', 'wrists']

Input Tags: [('gloves', 37), ('leather', 34), ('touchscreen-compatible', 4)]

#########################
#### RECOMMENDATIONS ####
#########################

1. Water-repellent, softshell gloves with a soft fleece lining and reinforced palms in imitation leather. Elastication at the wrists, reflective details and a plastic hook-and-eye fastening to keep the gloves together. The tops of the thumb and index finger are made from touchscreen-compatible material.


{'leather': 0.19889445263079186,
 'gloves': 0.15505318557230835,
 'touchscreen-compatible': 0.012014470486182064}

------------------------------------------------------------------------------------------------------------------------------------------------------
2. Leather gloves with the tops of the thumb and index finger in touchscreen-compatible material. Lined.


{'leather': 0.1614813873679882,
 'gloves': 0.1188826568163016,
 'touchscreen-compatible': 0.01029643639755287}

------------------------------------------------------------------------------------------------------------------------------------------------------
3. Leather gloves. The tops of the thumb, index finger and middle finger have been treated to make them touchscreen-compatible. Lined.


...

------------------------------------------------------------------------------------------------------------------------------------------------------
4. Fine-knit gloves with touchscreen-compatible material on the thumb, index and middle finger.


{'gloves': 0.1109595328657305, 'touchscreen-compatible': 0.007278754446910396}

------------------------------------------------------------------------------------------------------------------------------------------------------
5. Gloves in soft, supple leather. Lined.


{'leather': 0.21454706074265623, 'gloves': 0.13979451376818156}

------------------------------------------------------------------------------------------------------------------------------------------------------
